In [37]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [38]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [39]:
data = unpickle(r'C:\Users\seven\Downloads\cifar-10-batches-py\data_batch_1')
X = data[b'data']
Y = data[b'labels']
X.shape

(10000, 3072)

In [40]:
# X=np.reshape(X,(-1,32,32,3))
# X = X.astype(np.float32)
# X = X.transpose((0,3,1,2))
X=np.reshape(X,(-1,3,32,32))
X = X.astype(np.float32)
# X = X.transpose((0,3,1,2))
X.shape

(10000, 3, 32, 32)

In [41]:
from torch.utils.data import DataLoader

class my_dataset(torch.utils.data.Dataset):
    def __init__(self, ):
        pass

    def __getitem__(self, index):
        return X[index],Y[index]

    def __len__(self):
        return len(X)

ds=my_dataset()

In [42]:


from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader
# 划分训练和测试的数据
train, test = random_split(ds, (9000, 1000) )
# 设定每次读取的数量
data_loader = torch.utils.data.DataLoader(dataset=train,batch_size=200, shuffle=True)
data_loader2 = torch.utils.data.DataLoader(dataset=test,batch_size=1000, shuffle=True)

In [43]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.L1 = nn.Conv2d(3,32,7,2,1)
        self.B1= nn.BatchNorm2d(32)

        self.L2 = nn.Conv2d(32,64,5,2,2)
        self.B2= nn.BatchNorm2d(64)
        
        self.L3 = nn.Conv2d(64,128,3,1,1)
        self.B3= nn.BatchNorm2d(128)
        
        self.L = nn.Linear(6272,10)

    def forward(self, x):
        #降二维，从start_dim开始相乘
        # x = torch.flatten(x,start_dim = 1)
        g1 = self.L1(x)
        g1 = self.B1(g1)
        # f1=torch.sigmoid(g1)
        f1=torch.relu(g1)

        g2 = self.L2(f1)
        g2 = self.B2(g2)
        f2 = torch.relu(g2)

        g3 = self.L3(f2)
        g3 = self.B3(g3)
        f3 = torch.relu(g3)

        f3 = torch.flatten(f3,start_dim = 1)

        g = self.L(f3)

        return g
    
model = MyNet()
model = torch.load('model4.pth')

In [44]:
import torchmetrics

lossf = nn.CrossEntropyLoss()     #loss函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  #SGD函数
metricsf = torchmetrics.Accuracy(task='multiclass', num_classes=10)       #torchmetrics函数

for i in range(50):
    for batchX, batchY in data_loader:
        score = model(batchX)
        score = torch.squeeze(score)
        loss = lossf(score, batchY)

        loss.backward() #求导
        optimizer.step() #梯度下降
        optimizer.zero_grad() #梯度清零

        mse = metricsf(score, batchY)    
    print(loss, metricsf.compute())
    metricsf.reset()

tensor(1.7214, grad_fn=<NllLossBackward0>) tensor(0.3111)
tensor(1.6708, grad_fn=<NllLossBackward0>) tensor(0.4418)
tensor(1.4259, grad_fn=<NllLossBackward0>) tensor(0.5028)
tensor(1.4214, grad_fn=<NllLossBackward0>) tensor(0.5317)
tensor(1.2159, grad_fn=<NllLossBackward0>) tensor(0.5632)
tensor(1.1031, grad_fn=<NllLossBackward0>) tensor(0.5994)
tensor(1.1184, grad_fn=<NllLossBackward0>) tensor(0.6202)
tensor(1.2541, grad_fn=<NllLossBackward0>) tensor(0.6413)
tensor(0.9875, grad_fn=<NllLossBackward0>) tensor(0.6661)
tensor(0.8837, grad_fn=<NllLossBackward0>) tensor(0.6839)
tensor(0.9090, grad_fn=<NllLossBackward0>) tensor(0.7151)
tensor(0.8940, grad_fn=<NllLossBackward0>) tensor(0.7236)
tensor(0.8084, grad_fn=<NllLossBackward0>) tensor(0.7452)
tensor(0.8232, grad_fn=<NllLossBackward0>) tensor(0.7629)
tensor(0.7414, grad_fn=<NllLossBackward0>) tensor(0.7819)
tensor(0.6573, grad_fn=<NllLossBackward0>) tensor(0.7977)
tensor(0.6333, grad_fn=<NllLossBackward0>) tensor(0.8122)
tensor(0.6172,

In [45]:
torch.save(model, 'model4.pth')

In [47]:
model.eval()

for batchX, batchY in data_loader2:
    score = model(batchX)
    score = torch.squeeze(score)
    mse = metricsf(score, batchY)  
      
print(metricsf.compute())

tensor(0.5290)
